# Task for Today  

***

## Hand Movement and User Prediction  

Given *data EEG data from different users performing different hand movements*, let's try to predict the **hand movement** and the **user** of a given reading.

We will use a TensorFlow neural network to make our predictions. 

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [5]:
dfs = [pd.read_csv('../input/eeg-data-from-hands-movement/Dataset/user_' + user + '.csv') for user in ['a', 'b', 'c', 'd']]

In [6]:
for i in range(len(dfs)):
    dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [7]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)

In [8]:
data

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,55.547547,3604.601528,2122.493834,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,7.208052,3513.244789,2030.461207,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2.0,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,...,1.910036,3567.687654,2062.791757,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,3
11516,1.0,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,...,5.217772,3548.277991,2055.537892,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,2
11517,0.0,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,...,2.200203,3572.290285,2063.776063,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,1
11518,0.0,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,...,15.413625,3608.394990,2091.509508,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,0


# Helper Functions

In [26]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [11]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()
    
    # One-hot encode whichever target column is not being used
    targets = ['Class', 'User']
    targets.remove(target)
    df = onehot_encode(df, column=targets[0])
    
    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [12]:
def build_model(num_classes=3):
    
    inputs = tf.keras.Input(shape=(X_train.shape[1],))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Predicting Hand Movement Class

In [13]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')

In [14]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.574343,-0.578735,-0.575870,1.722792
1,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,1.741120,-0.578735,-0.575870,-0.580453
2,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.574343,-0.578735,-0.575870,1.722792
3,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,-0.574343,-0.578735,-0.575870,1.722792
4,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.574343,1.727908,-0.575870,-0.580453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.574343,-0.578735,1.736502,-0.580453
8059,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,1.741120,-0.578735,-0.575870,-0.580453
8060,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.574343,-0.578735,1.736502,-0.580453
8061,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.574343,1.727908,-0.575870,-0.580453


In [15]:
y_train

8701     1.0
4335     1.0
11203    1.0
11448    0.0
374      2.0
        ... 
9785     1.0
7763     2.0
5218     1.0
1346     1.0
3582     2.0
Name: Class, Length: 8063, dtype: float64

In [16]:
y_train.value_counts()

2.0    2716
0.0    2698
1.0    2649
Name: Class, dtype: int64

In [17]:
class_model = build_model(num_classes=3)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
202/202 [==============================] - 0s 2ms/step - loss: 1.0738 - accuracy: 0.4236 - val_loss: 1.0162 - val_accuracy: 0.4526
Epoch 2/50
202/202 [==============================] - 0s 1ms/step - loss: 0.9787 - accuracy: 0.5057 - val_loss: 1.0132 - val_accuracy: 0.4923
Epoch 3/50
202/202 [==============================] - 0s 1ms/step - loss: 0.9343 - accuracy: 0.5468 - val_loss: 0.9806 - val_accuracy: 0.5139
Epoch 4/50
202/202 [==============================] - 0s 1ms/step - loss: 0.8789 - accuracy: 0.5800 - val_loss: 0.9303 - val_accuracy: 0.5468
Epoch 5/50
202/202 [==============================] - 0s 1ms/step - loss: 0.8313 - accuracy: 0.6087 - val_loss: 0.9023 - val_accuracy: 0.5741
Epoch 6/50
202/202 [==============================] - 0s 1ms/step - loss: 0.7875 - accuracy: 0.6338 - val_loss: 0.8923 - val_accuracy: 0.5834
Epoch 7/50
202/202 [==============================] - 0s 1ms/step - loss: 0.7427 - accuracy: 0.6681 - val_loss: 0.8673 - val_accuracy: 0.5902
Epoch 

In [18]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 62.51%


# Predicting User

In [19]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='User')

In [20]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,Class_0.0,Class_1.0,Class_2.0
0,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.535664,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.709147,1.429612,-0.712705
1,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,3.858153,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,-0.709147,1.429612,-0.712705
2,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.537446,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.709147,1.429612,-0.712705
3,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,-0.382336,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,1.410145,-0.699491,-0.712705
4,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.465464,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.709147,-0.699491,1.403105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,1.371221,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.709147,1.429612,-0.712705
8059,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,1.627338,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,-0.709147,-0.699491,1.403105
8060,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.167387,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.709147,1.429612,-0.712705
8061,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.463766,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.709147,1.429612,-0.712705


In [21]:
y_train

8701     3
4335     0
11203    3
11448    3
374      1
        ..
9785     2
7763     0
5218     2
1346     1
3582     2
Name: User, Length: 8063, dtype: int64

In [22]:
y_train.value_counts()

3    2032
1    2023
2    2008
0    2000
Name: User, dtype: int64

In [23]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
202/202 [==============================] - 0s 2ms/step - loss: 0.1342 - accuracy: 0.9591 - val_loss: 0.0240 - val_accuracy: 0.9944
Epoch 2/50
202/202 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.9953 - val_loss: 0.0108 - val_accuracy: 0.9963
Epoch 3/50
202/202 [==============================] - 0s 1ms/step - loss: 0.0065 - accuracy: 0.9984 - val_loss: 0.0052 - val_accuracy: 0.9981
Epoch 4/50
202/202 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0042 - val_accuracy: 0.9994
Epoch 5/50
202/202 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0042 - val_accuracy: 0.9975
Epoch 6/50
202/202 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.0019 - val_accuracy: 0.9994
Epoch 7/50
202/202 [==============================] - 0s 1ms/step - loss: 8.0638e-04 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 0.9981
Ep

In [24]:
user_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (User Model): {:.2f}%".format(user_acc * 100))

Test Accuracy (User Model): 99.97%


# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/hEExwYfoieY